In [1]:
%load_ext autoreload  
%autoreload 2 

In [2]:
import sqlite3
from typing import Any, Optional
from pydantic import BaseModel, Extra, Field
from transformers import TrainingArguments, AutoModel, AutoTokenizer, AutoModelForSequenceClassification, get_scheduler
import numpy as np
import json
import datasets
import wandb
import torch
from tqdm import tqdm
from subset_active_learning.subset_selection import select, preprocess

/Users/garylai/Dev/subset-active-learning/.venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
DB_PATH = "/Users/garylai/Dev/subset-active-learning/scripts/subset_selection/new_sst.db"

training_args = select.SubsetTrainingArguments(eval_steps=3, max_steps=5)
searching_args = select.SubsetSearcherArguments(seed=0, db_path=DB_PATH)

In [6]:
processed_ds = preprocess.preprocess_sst2(training_args.model_card)

100%|██████████| 3/3 [00:00<00:00, 155.93it/s]


In [7]:
subset_trainer = select.SubsetTrainer(
    params=training_args, valid_ds=processed_ds["validation"], test_ds=processed_ds["test"]
)

In [8]:
data_pool = processed_ds["train"].shuffle(seed=searching_args.seed).select(range(searching_args.data_pool_size))

In [10]:
subset_searcher = select.SubsetSearcher(subset_trainer=subset_trainer, params=searching_args, data_pool=data_pool)

In [11]:
subset_searcher.search(n_runs=3)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: garylai (johnny-gary). Use `wandb login --relogin` to force relogin


Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier

data_pool_size,▁
loss,█▃█▁▃
optimal_subset_size,▁
sst2_test:accuracy,▁
sst:val_acc,▁
data_pool_size,1000
indices,"[814, 671, 49, 206, ..."
loss,0.6918
model_card,google/electra-small...
optimal_subset_size,100
sst2_test:accuracy,0.5


Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier

data_pool_size,▁
loss,█▃█▁▃
optimal_subset_size,▁
sst2_test:accuracy,▁
sst:val_acc,▁
data_pool_size,1000
indices,"[814, 671, 49, 206, ..."
loss,0.6918
model_card,google/electra-small...
optimal_subset_size,100
sst2_test:accuracy,0.5


Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier

In [12]:
from subset_active_learning.active_learning.subset_classifier import get_df_from_db
df = get_df_from_db(DB_PATH); df

,indexes,objective
0,"[814, 671, 49, 206, 832, 564, 400, 695, 943, 7...",0.000000
1,"[814, 671, 49, 206, 832, 564, 400, 695, 943, 7...",0.500000
2,"[814, 671, 49, 206, 832, 564, 400, 695, 659, 7...",0.500000
3,"[814, 671, 49, 206, 832, 564, 400, 695, 659, 7...",0.499548
